In [0]:
import pandas as pd
#KAGGLE YELP DATASET
# c_bus = ['business_id', 'name', 'neighborhood', 'address', 'city', 'state','postal_code', 'latitude', 'longitude', 'stars', 'review_count','is_open', 'categories']
bus = pd.read_csv("csv/yelp_business.csv")  
# c_bh = ['business_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday','saturday', 'sunday']
bh = pd.read_csv("csv/yelp_business_hours.csv") 
# c_check = ['business_id', 'weekday', 'hour', 'checkins']
check = pd.read_csv("csv/yelp_checkin.csv") 
#c_rev = ['review_id', 'user_id', 'business_id', 'stars', 'date', 'text','useful', 'funny', 'cool']
rev = pd.read_csv("csv/yelp_review.csv") 
ba = pd.read_csv("csv/yelp_business_attributes.csv")

In [0]:
#PROFESSOR YELPZIP DATASET
c_md = ['user_id','prod_id','rating','label','date']
c_pm = ['res_name','prod_id']
c_rc = ['user_id','prod_id','date','review']
md = pd.read_csv('labelcsv/metadata', sep='\t', names=c_md)
pm = pd.read_csv('labelcsv/productIdMapping',sep='\t', names=c_pm)
rc = pd.read_csv('labelcsv/reviewContent',sep='\t', names=c_rc)

DATASET 1 PREPROCESSING:

In [0]:
# labelled reviews
i = pd.merge(rc, md,  how='left', left_on=['user_id','prod_id','date'], right_on = ['user_id','prod_id','date'])
i.to_csv(r'revlabeled.csv')

In [0]:
#finding names that intersect between both the datasets (3266) 
bness = k.iloc[l]
name = bness['name'].tolist()
s1 = pd.Series(name).str.replace('"', '')
s2 = pm['res_name'].tolist()
final_names = set(s2).intersection(set(s1))

In [0]:
#businesses with restaurant names intersecting between both the datasets
p = pd.DataFrame(columns=['business_id','name'])
s = list(final_names)
s = ['"' + item + '"' for item in s]
for i,row in bness.iterrows():
    if row['name'] in s:
        p = p.append(row[['name','business_id']], ignore_index = True)
p.to_csv(r'intersection.csv')

DATASET 2 PREPROCESSING:

In [0]:
# Businesses with ['categories'=='Restaurant']
bus.head(20)
k = bus
b = k['categories'].tolist()
l = []
for i in range(len(b)):
    if 'Restaurants' in b[i]:
        l.append(i)

In [0]:
#all the business_id of the restaurants we are using
business = k.iloc[l]
bid = business['business_id'].tolist()

In [0]:
#keep rows of only restaurants for ba,bh
busat = ba[['business_id','BusinessParking_garage','BusinessParking_street', 'BusinessParking_validated','BusinessParking_lot', 'BusinessParking_valet','WheelchairAccessible','BikeParking','DogsAllowed','OutdoorSeating']]
b = pd.DataFrame(columns=busat.columns)
for i,row in busat.iterrows():
    if row['business_id'] in bid:
        b = b.append(row, ignore_index = True)
h = pd.DataFrame(columns=bh.columns)
for i,row in bh.iterrows():
    if row['business_id'] in bid:
        h = h.append(row, ignore_index = True)

In [0]:
#combine all attributes into one business attributes file
dfs = [business,b,h]
dfs = [df.set_index('business_id') for df in dfs]
d = dfs[0].join(dfs[1:])
d.to_csv(r'business.csv')

In [0]:
import pandas as pd
import numpy as np
bus = pd.read_csv("business.csv")  

In [0]:
bus = bus.drop(['neighborhood', 'address', 'postal_code', 'latitude', 'longitude'], axis=1)

In [0]:
p = bus[['BusinessParking_garage','BusinessParking_street', 'BusinessParking_validated',
       'BusinessParking_lot', 'BusinessParking_valet','BikeParking']]
p.head()

,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,BikeParking
0,Na,Na,Na,Na,Na,Na
1,Na,False,False,False,False,False
2,Na,False,False,False,True,True
3,Na,Na,Na,Na,Na,Na
4,Na,False,True,False,False,False


In [0]:
p = p.replace(to_replace = "Na", value =np.nan)
p = p.replace(to_replace = "False", value =np.nan)
p['BusinessParking_garage'].value_counts()

True    90
Name: BusinessParking_garage, dtype: int64

In [0]:
def my_function(row):
    return all(row[['BusinessParking_garage','BusinessParking_street', 'BusinessParking_validated',
       'BusinessParking_lot', 'BusinessParking_valet','BikeParking']].isna())
p['parking'] = p.apply(my_function, axis=1)

In [0]:
p['parking'] = p['parking'].replace(to_replace = True, value = "No")
p['parking'] = p['parking'].replace(to_replace = False, value = "Yes")
p['parking'].value_counts()

No     28656
Yes    25962
Name: parking, dtype: int64

In [0]:
bus = bus.drop(['BusinessParking_garage','BusinessParking_street', 'BusinessParking_validated',
       'BusinessParking_lot', 'BusinessParking_valet','BikeParking'], axis=1)

In [0]:
bus['parking'] = p['parking']

In [0]:
l = ['American (New)','Nightlife','Bars','Sandwiches','American (Traditional)','Burgers','Restaurants','Italian','Breakfast & Brunch','Gluten-Free','Coffee & Tea','Food','French','Bakeries','Mexican','Gastropubs','Mediterranean','Tea Rooms','Izakaya','Comfort Food','Cocktail Bars','Asian Fusion','Japanese','Seafood','Fast Food','Barbeque','Sports Bars','Chinese','Cafes','Diners','British','Bagels','Pizza','Dive Bars','Imported Food','Specialty Food','Thai','Ethnic Food','Food Court','Cuban','Indian','Internet Cafes','Caribbean','Canadian (New)','Chicken Wings','Tacos','Food Trucks','Vegan','Wraps','Buffets','Sushi Bars','Wine Bars','Donuts','Delis','Vietnamese','German','Creperies','Delicatessen','Local Flavor','Tex-Mex','Hotels','Greek','Ramen','Hot Dogs','Dim Sum','Caterers','Food Delivery Services','Noodles','Karaoke','Dance Clubs','Beer','Wine & Spirits','Belgian','Pubs','Vegetarian','International','Modern European','Steakhouses','Bubble Tea','Cantonese','Korean','Southern','Hungarian','Whiskey Bars','Ice Cream & Frozen Yogurt','Latin American','Pakistani','Halal','Desserts','Fish & Chips','Poutineries','Candy Stores','Irish','Middle Eastern','Tapas Bars','Himalayan/Nepalese','Cajun/Creole','Live/Raw Food','Breweries','Hawaiian','Casinos','Bed & Breakfast','Afghan','Food Stands','Street Vendors','Ethiopian','Russian','African','Egyptian','Taiwanese','Brasseries','Scottish','Malaysian','Irish Pub','Kosher','Szechuan','Portuguese','Hakka','Bistros','Juice Bars & Smoothies','Persian/Iranian','Kebab','Beer Bar','Swabian','Cupcakes','Custom Cakes','Filipino','Eatertainment','Pan Asian','Gay Bars','Lebanese','Turkish','Polish','Salvadoran','Poke','Spanish','Brazilian','Beer Garden','Hot Pot','Fondue','Mongolian','Chocolatiers & Shops','Puerto Rican','Soul Food','Donairs','Arabian','Falafel','Gelato','Argentine','Coffee Roasteries','Beer Gardens','Patisserie/Cake Shop','Waffles','South African','Smokehouse','Hookah Bars','Peruvian','Venezuelan','Meat Shops','Teppanyaki','Basque','Cafeteria','Bangladeshi','Fruits & Veggies','Brewpubs','Pasta Shops','Butcher','New Mexican Cuisine','Singaporean','Indonesian','Cambodian','Wineries','Sri Lankan','Colombian','Scandinavian','Moroccan','Dominican','Ukrainian','Serbo Croatian','Austrian','Burmese','Laotian','Bavarian']

In [0]:
for i in l:
     bus[i] = 0

In [0]:
s = 'category_'
for index, row in bus.iterrows():
    for i in l:
        if i in row['categories']:
            bus.at[index, s+i] = 1
        else:
            bus.at[index, s+i] = 0

In [0]:
# bus = pd.concat([bus,pd.get_dummies(bus['location'],prefix='location')],axis=1)

In [0]:
bus['location'] = bus[['city', 'state']].apply(lambda x: ', '.join(x), axis=1)

In [0]:
bus = bus.drop(['city','state','categories','location'], axis=1)

In [0]:
bus['WheelchairAccessible'] = bus['WheelchairAccessible'].replace(to_replace = "False", value = 0)
bus['WheelchairAccessible'] = bus['WheelchairAccessible'].replace(to_replace = "Na", value = 0)
bus['WheelchairAccessible'] = bus['WheelchairAccessible'].replace(to_replace = "True", value = 1)
bus['WheelchairAccessible'] = bus['WheelchairAccessible'].replace(to_replace = np.nan, value = 0)

bus['DogsAllowed'] = bus['DogsAllowed'].replace(to_replace = "False", value = 0)
bus['DogsAllowed'] = bus['DogsAllowed'].replace(to_replace = "Na", value = 0)
bus['DogsAllowed'] = bus['DogsAllowed'].replace(to_replace = "True", value = 1)
bus['DogsAllowed'] = bus['DogsAllowed'].replace(to_replace = np.nan, value = 0)

bus['OutdoorSeating'] = bus['OutdoorSeating'].replace(to_replace = "False", value = 0)
bus['OutdoorSeating'] = bus['OutdoorSeating'].replace(to_replace = "Na", value = 0)
bus['OutdoorSeating'] = bus['OutdoorSeating'].replace(to_replace = "True", value = 1)
bus['OutdoorSeating'] = bus['OutdoorSeating'].replace(to_replace = np.nan, value = 0)

In [0]:
bus.to_csv(r'business2.csv')

DATASET 3 PREPROCESING:

In [0]:
rev = pd.read_csv("csv/yelp_review.csv") 
ba = pd.read_csv("csv/yelp_business.csv")

# Businesses with ['categories'=='Restaurant']
# ba.head(20)
k = ba
b = k['categories'].tolist()
l = []
for i in range(len(b)):
    if 'Restaurants' in b[i]:
        l.append(i)

In [0]:
rev = rev.drop(['user_id','useful','funny','cool','review_id'],axis=1)

In [0]:
business = k.iloc[l]
bid = business['business_id'].tolist()

In [0]:
#review file with only restaurants (without labels)
r = pd.DataFrame(columns=rev.columns)
for i,row in rev.iterrows():
    if row['business_id'] in bid:
        r = r.append(row, ignore_index = True)
r.to_csv(r'reviews.csv')

In [0]:
d = {}
for i, row in r.iterrows():
    if row['business_id'] not in d:
        d[row['business_id']] = 1
    else:
        d[row['business_id']] += 1

In [0]:
# l = []
# for key,value in d.items():
#     if value > 500:
#         l.append(key)

In [0]:
l = list(d.keys())

In [0]:
rev = pd.DataFrame(columns=r.columns)
for i,row in r.iterrows():
    if row['business_id'] in l:
        rev = rev.append(row, ignore_index = True)

In [0]:
d = dict.fromkeys(l,0)
bus = pd.read_csv("csv/yelp_business.csv")
for i,row in bus.iterrows():
    if row['business_id'] in d and d[row['business_id']]==0:
        d[row['business_id']] = (row['name']).replace('"', '')

In [0]:
rev['res_name'] = rev['business_id']
for i,row in rev.iterrows():
    row['res_name'] = d[row['business_id']]

In [0]:
rev = rev[['business_id', 'res_name', 'date', 'text', 'stars']]

In [0]:
rev['res_name'].replace(d, inplace=True)